We'll download the dataset first:

In [1]:
from tensorflow import keras

(X_train, Y_train), (X_validation, Y_validation) = \
    keras.datasets.cifar10.load_data()

Y_train = keras.utils.to_categorical(Y_train, 10)
Y_validation = keras.utils.to_categorical(Y_validation, 10)

170498071/170498071 [==============================] - 14s 0us/step


Next, we'll instantiate a DataGenerator, which will normalize the train and test datasets, and will provide data augmentation during training:

In [2]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True)

# Apply z-normalization on the training set
data_generator.fit(X_train)

# Standardize the validation set
X_validation = data_generator.standardize(X_validation.astype('float32'))

Then, we'll define the CNN model:

In [3]:
from keras import layers, models

model = models.Sequential(layers=[
    layers.Conv2D(32, (3, 3),
                  padding='same',
                  input_shape=X_train.shape[1:]),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),

    layers.Conv2D(64, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Conv2D(64, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3)),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Conv2D(128, (3, 3)),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

We'll define the optimization parameters:

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0

Finally, we'll run the training and evaluation:

In [5]:
batch_size = 50

model.fit(
    x=data_generator.flow(x=X_train,
                          y=Y_train,
                          batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=100,
    verbose=1,
    validation_data=(X_validation, Y_validation),
    workers=4)

Epoch 1/100
1000/1000 [==============================] - 24s 16ms/step - loss: 1.6647 - accuracy: 0.4083 - val_loss: 1.2007 - val_accuracy: 0.5581
Epoch 2/100
1000/1000 [==============================] - 15s 15ms/step - loss: 1.2007 - accuracy: 0.5688 - val_loss: 1.0213 - val_accuracy: 0.6286
Epoch 3/100
1000/1000 [==============================] - 16s 16ms/step - loss: 1.0357 - accuracy: 0.6273 - val_loss: 0.8928 - val_accuracy: 0.6804
Epoch 4/100
1000/1000 [==============================] - 15s 15ms/step - loss: 0.9370 - accuracy: 0.6658 - val_loss: 0.9065 - val_accuracy: 0.6760
Epoch 5/100
1000/1000 [==============================] - 15s 15ms/step - loss: 0.8691 - accuracy: 0.6937 - val_loss: 0.7831 - val_accuracy: 0.7200
Epoch 6/100
1000/1000 [==============================] - 15s 15ms/step - loss: 0.8132 - accuracy: 0.7108 - val_loss: 0.7412 - val_accuracy: 0.7422
Epoch 7/100
1000/1000 [==============================] - 16s 16ms/step - loss: 0.7774 - accuracy: 0.7272 - val_loss: 0